In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import tensorflow as tf
from tensorflow.python.ops import control_flow_ops
import os, os.path
import sys
sys.path.insert(0,'../processing/')
sys.path.insert(0,'../')
from datasets import sythtextprovider
import load_batch
from nets import txtbox_300, textbox_common, np_methods
#from processing import image_processing
from image_processing2 import *
from processing import txt_preprocessing
import tf_utils
import time
slim = tf.contrib.slim
import load_batch
import numpy as np
import skimage.io as skio
from processing import tf_image
import cv2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
isess = tf.InteractiveSession()

In [10]:
with tf.Graph().as_default(): 
    text_net = txtbox_300.TextboxNet()
    text_shape = text_net.params.img_shape
    print 'text_shape '+  str(text_shape)
    text_anchors = text_net.anchors(text_shape)

    b_image, b_glocalisations, b_gscores = \
        load_batch.get_batch('../data/sythtext/',
                             3,
                             1,
                             text_shape,
                             text_net,
                             text_anchors,
                             2,
                             file_pattern = '*.tfrecord',
                             is_training = True,
                             shuffe = False)
    localisations, logits, end_points = \
                        text_net.net(b_image, is_training=True, use_batch=False)
    with tf.Session() as sess: 
        sess.run(tf.global_variables_initializer())
        with slim.queues.QueueRunners(sess):
            for i in xrange(1):
                

text_shape (300, 300)
file_path: ../data/sythtext/*.tfrecord


In [40]:
gradient_multipliers = {
            'conv1/conv1_1/weights_1' : 0.,
            'conv1/conv1_1/biases_1' : 0.,
            'conv1/conv1_2/weights_1' : 0.,
            'conv1/conv1_2/biases_1' : 0.,
            'conv2/conv2_1/weights_1' : 0.,
            'conv2/conv2_1/biases_1' : 0.,
            'conv2/conv2_2/weights_1' : 0.,
            'conv2/conv2_2/biases_1' : 0.,
            'conv3/conv3_1/weights_1' : 1.,
            'conv3/conv3_1/biases_1' : 2.,
            'conv3/conv3_2/weights_1' : 1.,
            'conv3/conv3_2/biases_1' : 2.,
            'conv4/conv4_1/weights_1' : 1.,
            'conv4/conv4_1/biases_1' : 2.,
            'conv4/conv4_2/weights_1' : 1.,
            'conv4/conv4_2/biases_1' : 2.,
            'conv4/conv4_3/weights_1' : 1.,
            'conv4/conv4_3/biases_1' : 2.,
            'conv5/conv5_1/weights_1' : 1.,
            'conv5/conv5_1/biases_1' : 2.,
            'conv5/conv5_2/weights_1' : 1.,
            'conv5/conv5_2/biases_1' : 2.,
            'conv5/conv5_3/weights_1' : 1.,
            'conv5/conv5_3/biases_1' : 2.,
            'conv6/weights_1' : 1.,
            'conv6/biases_1' : 2.,
            'conv7/weights_1' : 1.,
            'conv7/biases_1' : 2.,
            'conv8/conv1x1/weights_1' :1.,
            'conv8/conv1x1/biases_1' :2.,
            'conv8/conv3x3/weights_1' :1.,
            'conv8/conv3x3/biases_1' :2.,
            'conv9/conv1x1/weights_1' :1.,
            'conv9/conv1x1/biases_1' :2.,
            'conv9/conv3x3/weights_1' :1.,
            'conv9/conv3x3/biases_1' :2.,
            'conv10/conv1x1/weights_1' :1.,
            'conv10/conv1x1/biases_1' :2.,
            'conv10/conv3x3/weights_1' :1.,
            'conv10/conv3x3/biases_1' :2.,
            'global/conv1x1/weights_1' :1.,
            'global/conv1x1/biases_1' :2.,
            'global/conv3x3/weights_1' :1.,
            'global/conv3x3/biases_1' :2.,
            'conv4_box/conv_cls/weights_1': 1.,
            'conv4_box/conv_cls/biases_1': 2.,
            'conv4_box/conv_loc/weights_1': 1.,
            'conv4_box/conv_loc/biases_1': 2.,
            'conv7_box/conv_loc/weights_1': 1.,
            'conv7_box/conv_loc/biases_1': 2.,
            'conv7_box/conv_cls/weights_1': 1.,
            'conv7_box/conv_cls/biases_1': 2.,
            'conv8_box/conv_loc/weights_1': 1.,
            'conv8_box/conv_loc/biases_1': 2.,
            'conv8_box/conv_cls/weights_1': 1.,
            'conv8_box/conv_cls/biases_1': 2.,
            'conv9_box/conv_loc/weights_1': 1.,
            'conv9_box/conv_loc/biases_1': 2.,
            'conv9_box/conv_cls/weights_1': 1.,
            'conv9_box/conv_cls/biases_1': 2.,
            'conv10_box/conv_cls/weights_1': 1.,
            'conv10_box/conv_cls/biases_1': 2.,
            'conv10_box/conv_loc/weights_1': 1.,
            'conv10_box/conv_loc/biases_1': 2.,
            'global_box/conv_cls/weights_1': 1.,
            'global_box/conv_cls/biases_1': 2.,
            'global_box/conv_loc/weights_1': 1.,
            'global_box/conv_loc/biases_1': 2.
        }

In [41]:
#text_box_300/conv1/conv1_1/weights
#text_box_300/conv2/conv2_1/biases
gradient_multipliers = {'text_box_300/'+k:v for k,v in gradient_multipliers.items()}
import pickle
pickle.dump( gradient_multipliers, open('../nets/multiplier_300.pkl','wb'),)

In [42]:
pickle.load(open('../nets/multiplier_300.pkl','rb'))

{'text_box_300/conv1/conv1_1/biases_1': 0.0,
 'text_box_300/conv1/conv1_1/weights_1': 0.0,
 'text_box_300/conv1/conv1_2/biases_1': 0.0,
 'text_box_300/conv1/conv1_2/weights_1': 0.0,
 'text_box_300/conv10/conv1x1/biases_1': 2.0,
 'text_box_300/conv10/conv1x1/weights_1': 1.0,
 'text_box_300/conv10/conv3x3/biases_1': 2.0,
 'text_box_300/conv10/conv3x3/weights_1': 1.0,
 'text_box_300/conv10_box/conv_cls/biases_1': 2.0,
 'text_box_300/conv10_box/conv_cls/weights_1': 1.0,
 'text_box_300/conv10_box/conv_loc/biases_1': 2.0,
 'text_box_300/conv10_box/conv_loc/weights_1': 1.0,
 'text_box_300/conv2/conv2_1/biases_1': 0.0,
 'text_box_300/conv2/conv2_1/weights_1': 0.0,
 'text_box_300/conv2/conv2_2/biases_1': 0.0,
 'text_box_300/conv2/conv2_2/weights_1': 0.0,
 'text_box_300/conv3/conv3_1/biases_1': 2.0,
 'text_box_300/conv3/conv3_1/weights_1': 1.0,
 'text_box_300/conv3/conv3_2/biases_1': 2.0,
 'text_box_300/conv3/conv3_2/weights_1': 1.0,
 'text_box_300/conv4/conv4_1/biases_1': 2.0,
 'text_box_300/co